In [ ]:
from structure import World
import pandas as pd
import numpy as np
from streamz.dataframe import PeriodicDataFrame
from time import sleep
from IPython.display import display
import ipywidgets as widgets
from threading import Thread
from ipywidgets import interact

In [ ]:
def get_statistics_report(results):
    sum_steps = 0
    n_worlds = 0
    
    min_step = np.inf
    world_min_step = None
    
    max_step = 0
    world_max_step = None
    
    sum_punctuation = 0
    n_ants = 0
    max_punctuation = 0
    
    for world_num, (step, winner, x, y, food_results, colony_results) in enumerate(results):   
        sum_steps += step
        n_worlds += 1

        if step < min_step:
            min_step = step
            world_min_step = world_num

        if step > max_step:
            max_step = step
            world_max_step = world_num

        for col_num, x, y, amount_of_food, ant_results, pheromone_results in colony_results:
            for ant_num, x, y, old_x, old_y, has_food, punctuation in ant_results:
                sum_punctuation += punctuation
                n_ants += 1
                max_punctuation = max(max_punctuation, punctuation)
                    
    report = pd.DataFrame({'Values': [sum_steps/n_worlds, world_min_step, min_step, world_max_step, max_step, sum_punctuation/n_ants, max_punctuation]}, index=['mean_steps', 'world_min_step', 'min_step', 'world_max_step', 'max_step', 'mean_pontuation', 'max_punctuation'])
    
    return report

In [ ]:
def get_global_report(results):
    report = pd.DataFrame({'step': [0 for _ in range(n_worlds+1)],
                           'n_cols': [0 for _ in range(n_worlds+1)],
                           'n_ants': [0 for _ in range(n_worlds+1)], 
                           'n_ants_w_food': [0 for _ in range(n_worlds+1)], 
                           'n_food_in_cols': [0 for _ in range(n_worlds+1)], 
                           'n_food_in_foods': [0 for _ in range(n_worlds+1)]},
                           index=pd.Index(data=[i for i in range(n_worlds)] + ['total'], name='world'))

    for world_num, (step, winner, x, y, food_results, colony_results) in enumerate(results):
        report['step'][world_num] = step

        for col_num, x, y, amount_of_food, ant_results, pheromone_results in colony_results:
            report['n_cols'][world_num] += 1
            report['n_food_in_cols'][world_num] += amount_of_food

            for ant_num, x, y, old_x, old_y, has_food, punctuation in ant_results:
                report['n_ants'][world_num] += 1

                if has_food:
                    report['n_ants_w_food'][world_num] += 1

        for num, x, y, amount, inside, limit in food_results:
            report['n_food_in_foods'][world_num] += amount
            
    report.loc['total'] = report.iloc[:n_worlds].sum(axis=0)
    
    total_ants = report['n_ants']
    report['%_ants_w_food'] = (100 * report['n_ants_w_food'] / total_ants).astype(np.int64)
    report['%_ants_look_food'] = 100 - report['%_ants_w_food']
    
    total_food = report['n_ants_w_food'] + report['n_food_in_cols'] + report['n_food_in_foods']
    report['%_food_in_trans'] = (100 * report['n_ants_w_food'] / total_food).fillna(0).astype(np.int64)
    report['%_food_in_cols'] = (100 * report['n_food_in_cols'] / total_food).fillna(0).astype(np.int64)
    report['%_food_in_foods'] = (100 * report['n_food_in_foods'] / total_food).fillna(0).astype(np.int64)
    
    return report

In [ ]:
def get_world_report(result):
    step, winner, x, y, food_results, colony_results = result

    n_cols = len(colony_results)
    report = pd.DataFrame({'n_ants': [0 for _ in range(n_cols+1)],
                           'n_foods': [0 for _ in range(n_cols+1)],
                           'n_ants_w_food': [0 for _ in range(n_cols+1)]},
                           index=pd.Index(data=[i for i in range(n_cols)] + ['total'], name='colony'))

    for col_num, x, y, amount_of_food, ant_results, pheromone_results in colony_results:
        for ant_num, x, y, old_x, old_y, has_food, punctuation in ant_results:
            report['n_ants'][col_num] += 1
            
            if has_food:
                report['n_ants_w_food'][col_num] += 1

        report['n_foods'][col_num] = amount_of_food

    report.loc['total'] = report.iloc[:n_cols].sum(axis=0)
    
    total_ants = report['n_ants']
    report['%_ants_w_food'] = (100 * report['n_ants_w_food'] / total_ants).astype(np.int64)
    report['%_ants_look_food'] = 100 - report['%_ants_w_food']
    
    total_food = report['n_foods']
    report['%_food_in_trans'] = (100 * report['n_ants_w_food'] / (report['n_ants_w_food'] + report['n_foods'])).fillna(0).astype(np.int64)
    report['%_food_in_cols'] = 100 - report['%_food_in_trans']
    
    report['prob_of_win'] = (100 * report['n_foods'] / report['n_foods'].iloc[:n_cols].sum()).fillna(0).astype(np.int64)
    
    return report

In [ ]:
sleeptime = 0.5
n_worlds = 5
report_type = 'Statistics'

In [ ]:
report = pd.DataFrame()
view = PeriodicDataFrame(lambda **kwargs: report, interval=sleeptime)

In [ ]:
widget = widgets.Dropdown(
    options=['Statistics', 'Global'] + [f'World {i}' for i in range(n_worlds)],
    value='Statistics',
    description='View'
)

def on_change(change):
    global report_type
    
    if change['type'] == 'change' and change['name'] == 'value':
        report_type = change['new']

widget.observe(on_change)
    
def display_widget():
    display(widget)
    display(view)

In [ ]:
def run_worlds(n=1000):
    global report, report_type
    
    worlds = [World(1000) for _ in range(n_worlds)]

    for _ in range(n):
        results = [world.update() for world in worlds]

        if report_type == 'Statistics':
            report = get_statistics_report(results)
        elif report_type == 'Global':
            report = get_global_report(results)
        else:
            report = get_world_report(results[int(report_type.split()[1])])

        sleep(sleeptime)

In [ ]:
Thread(target = run_worlds).start()
display_widget()